Exploring Image segmentation to inculde in workflow for delineating growing season irrigated areas

### libraries

In [1]:
from osgeo import gdal, ogr
import os
import gdal
import rsgislib
from rsgislib.segmentation import segutils
from rsgislib.rastergis import ratutils
from rasterstats import zonal_stats
from descartes import PolygonPatch
import matplotlib.cm as cm
import matplotlib.colors as colors
import fiona
import json
from fiona.crs import from_epsg
from shapely.geometry import Polygon, shape

## User inputs

In [79]:
year = '2017'

AOI = 'tamworth'

# This is the .tif file to be segmented
InputNDVIStats = 'data/' + AOI + '_201601_201712_MaxGNDVI.tif'

# These are interim outputs for this workflow
KEAFile = 'results/imageSeg/' + AOI + '_' + year + '.kea'
SegmentedKEAFile = 'results/imageSeg/' + AOI + '_' + year + '_sheperdSEG.kea'
SegmentedTiffFile = 'results/imageSeg/' + AOI + '_' + year + '_sheperdSEG.tif'
SegmentedPolygons = 'results/imageSeg/' + AOI + '_' + year + '_polygons.shp'

## image segmentation and format conversions

In [4]:
#convert tiff to .kea using gdal
os.system("gdal_translate -of KEA -a_srs EPSG:3577 " + InputNDVIStats + " " + KEAFile)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f92b2b86f00> >

In [56]:
#run the image segmentation code
segutils.runShepherdSegmentation(KEAFile, SegmentedKEAFile, numClusters=20, minPxls = 200)

Stretch Input Image
Add 1 to stretched file to ensure there are no all zeros (i.e., no data) regions created.
Create Input Image Mask.
Mask stretched Image.
Deleting file: ./tamworth_2017_stchdonly.kea
Deleting file: ./tamworth_2017_stchdonlyOff.kea
Deleting file: ./tamworth_2017_stchdmaskonly.kea
Performing KMeans.
Apply KMeans to image.
Eliminate Single Pixels.
Perform clump.
Eliminate small pixels.
Relabel clumps.
Calculate image statistics and build pyramids.
Deleting file: ./tamworth_2017_kmeansclusters.gmtxt
Deleting file: ./tamworth_2017_kmeans.kea
Deleting file: ./tamworth_2017_kmeans.kea.aux.xml
Deleting file: ./tamworth_2017_kmeans_nosgl.kea
Deleting file: ./tamworth_2017_kmeans_nosglTMP.kea
Deleting file: ./tamworth_2017_clumps.kea
Deleting file: ./tamworth_2017_clumps_elim.kea
Deleting file: ./tamworth_2017_stchd.kea


In [58]:
#convert imageSeg.kea file back into tiff
os.system("gdal_translate -of GTIFF -a_srs EPSG:3577 " + SegmentedKEAFile + " " + SegmentedTiffFile)

0

In [80]:
#turn tiff into a polygon set
os.system('gdal_polygonize.py ' + SegmentedTiffFile + ' -f' + ' ' + '"ESRI Shapefile"' + ' ' + SegmentedPolygons)

0

In [ ]:
# gdal.Translate(KEAFile, srcDS=InputNDVIStats, format="KEA", outputSRS = 'EPSG:3577')

# gdal.Translate(SegmentedTiffFile, srcDS=SegmentedKEAFile, format="GTiff", outputSRS = 'EPSG:3577')

# #  get raster datasource
# src_ds = gdal.Open(SegmentedTiffFile)
# src_band = src_ds.GetRasterBand(1)

# #  create output datasource
# dst_layername = SegmentedPolygons
# drv = ogr.GetDriverByName("ESRI Shapefile")
# dst_ds = drv.CreateDataSource(dst_layername)
# dst_layer = dst_ds.CreateLayer(dst_layername, srs = 'EPSG:3577' )

# gdal.Polygonize(src_band, None, dst_layer, -1, [])